<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2022TeamA/blob/main/COVPAC_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Vorbereitungen**

##Löschen aller Variablen

In [ ]:
%reset -f


## Import der Libraries

In [ ]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas_profiling import ProfileReport
import datetime

## Versions Check

In [ ]:
# Python
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
sns.__version__

In [ ]:
np.__version__

In [ ]:
np.__version__

#**Erstellen der Quelldatenbank**

In [ ]:
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2022TeamA/main/Pythonscripte/Quelldatenbank.py').text)

#**Erstellen des Datawarehouse**

In [ ]:
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2022TeamA/main/Pythonscripte/Datawarehouse.py').text)

# **Einfluss von COVID auf durchgeführte Prozeduren**

### Erstellen eines Ursprungsdataframes aus dem Datawarehouse 

In [ ]:
def connect_to_db(db_file):
    sqlite3_conn = None
    try:
        sqlite3_conn = sq.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()
  
conn_dwh = sqlite3.connect('/content/Datawarehouse.db')


In [ ]:
# Zentrum Tabelle
df_Zentrum = pd.read_sql_query("SELECT * FROM Zentrum", conn_dwh)

# demographic data
df_patients = pd.read_sql_query("SELECT * FROM dimPatients", conn_dwh)
# diagnoses data
df_conditions = pd.read_sql_query("select * from dimConditions", conn_dwh)

# procedure data
# df_careplans = pd.read_sql_query("select * from careplans_info", conn_dwh)
df_procedures = pd.read_sql_query("SELECT * FROM dimProcedures", conn_dwh)

# observations
df_observations = pd.read_sql_query("SELECT * FROM dimObservations", conn_dwh)

# encounters
df_encounters = pd.read_sql_query("SELECT * FROM dimEncounters", conn_dwh)

In [ ]:
# Join zwischen dimPatients und dimProcedures
covid_procedures = pd.read_sql_query("SELECT dimPatients.dataset_origin, dimProcedures.* FROM dimProcedures LEFT JOIN dimPatients ON dimPatients.id=dimProcedures.patient_id", conn_dwh)

In [ ]:
covid_procedures.head()

#### Hash-Check

In [ ]:
from pandas.util import hash_pandas_object
procedureshashes = hash_pandas_object(covid_procedures)


In [ ]:
procedureshashes

In [ ]:
#Wert sollte -8927845321861695504 sein 
procedureshashes.sum()

## Datenexploration

In [ ]:
# Nummer der Patienten in der Procedures Tabelle

In [ ]:
# Entfernen des Einträge aus dem Covid-19 Datensatz
final_df = covid_procedures[~covid_procedures['DATASET_ORIGIN'].isin(['covid19'])]

In [ ]:
final_df.head()

In [ ]:
# Nummer der Patienten in final_df
final_df.groupby(["PATIENT_ID"]).size()

In [ ]:
final_df.groupby(["DESCRIPTION"]).size()

In [ ]:
final_df.shape

In [ ]:
final_df.groupby(["PATIENT_ID", "CODE"]).size()

In [ ]:
final_df.groupby(["DATE"]).size()

## Datencleaning

In [ ]:
# Ersetzen von Nullwerten
final_df = final_df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# Anzahl der Nullwerte
final_df.isnull().sum()

In [ ]:
# Entfernen von Duplikaten mit Behalten des ersten Wertes
final_df.drop_duplicates(keep="first",inplace=True) 
print("Size of dataset after removinf duplicated rows", final_df.shape)

In [ ]:
# Attributinfo
final_df.info()

In [ ]:
# Ändern des Attribut-Typs
final_df["DATE"] = pd.to_datetime(final_df["DATE"])

In [ ]:
# Ändern des Datum-Typs zu YYYY
final_df["DATE"] = final_df["DATE"].dt.to_period('Y')

In [ ]:
final_df.head()

## Analyse der Prozuren nach Jahr

In [ ]:
# Prozedurtabelle währen COVID
covid_year = final_df[final_df["DATE"] >= "2020"]
covid_year.head()

In [ ]:
# Prozedurentabelle vor COVID
pre_covid = final_df[final_df["DATE"] < "2020"]
pre_covid.head()

In [ ]:
# Am häufigsten durchgeführte Prozeduren während Covid
covid_year['DESCRIPTION'].value_counts()[:20]

In [ ]:
# Am häufigsten durchgeführte Prozeduren Pre-Covid
pre_covid['DESCRIPTION'].value_counts()[:20]

## Visualisierung

In [ ]:
final_df.sort_values(by=['DATE'])

In [ ]:
# Auswertung der Prozeduren ab dem Jahr 2010
df_recent = final_df[final_df["DATE"] >= "2010"]

In [ ]:
df_recent = df_recent.sort_values(by='DATE')

In [ ]:
%matplotlib inline
sns.set(rc={'figure.figsize':(8, 8)})
sns.countplot(x='DATE', data=df_recent)

# **Vorhersage des Covid-Status anhand von Laborparametern**

## Erstellen eines Ursprungsdataframes aus dem Datawarehouse




In [ ]:
df = pd.read_sql_query("""
select t1.*, dimObservations.code, dimObservations.description, dimObservations.value, dimObservations.units, dimObservations.date from dimObservations
left JOIN
(select Zentrum.observation_id as OBSERVATION_ID, Zentrum.patient_id, dimPatients.dataset_origin from Zentrum, dimPatients where Zentrum.patient_id = dimPatients.id) t1
on dimObservations.id = t1.observation_id
;""", conn_dwh
  )

### Hash-Check

In [ ]:
from pandas.util import hash_pandas_object
dfhashes = hash_pandas_object(df)

In [ ]:
dfhashes

In [ ]:
#Wert sollte -8232134386015511732 sein 
dfhashes.sum()


## Datenexploration & Cleaning

In [ ]:
df.head(3)

In [ ]:
#Entfernen von Duplikaten
df.drop_duplicates(keep="first",inplace=True) 
print("Size of dataset after removing duplicated rows", df.shape)

In [ ]:
# Übersicht 
df.nunique(axis=0)

In [ ]:
#Ursprung der Daten
df.groupby(["dataset_origin"]).size()

## Datenvorbereitung & Cleaning

### Erstellen eines Dataframes, der nur Angaben zu COVID-positiven Patienten enthält





In [ ]:
#Zunächst Selektion der positiven Testergebnisse
dfpostest = df.loc[(df["CODE"] == "94531-1") & (df["VALUE"] == "Detected (qualifier value)")] 
dfpostest.head(3)



In [ ]:
# Erstellen eines Dataframes mit allen Laborwerten der Patienten mit positivem Testergebnis
dfpos = df.loc[((df.patient_id.isin(dfpostest['patient_id'])))]


In [ ]:
# Löschen der Patienten mit COVID-Nebendiagnse aus dem ALLERGY-Dataset, da diese zahlreiche nicht mit der COVID-Infektion im Zusammenhang stehende (ältere) Laborparameter haben
dfpos = dfpos.drop( dfpos[ dfpos['dataset_origin'] == "allergy" ].index)

In [ ]:
# Hinzufügen einer zusätzlichen Spalte mit dem Wert "1"
dfpos['Covid'] = '1'
dfpos.head(3)

In [ ]:
#Übersicht der Covid-Kohorte
dfpos.nunique(axis=0)

### Erstellen eines Dataframes, der nur die Patienten der Kontrollgruppe beinhaltet

In [ ]:
#Selektion der Laborparameter von Patienten ohne positiven COVID-Test aus dem Ursprungsdataframe
dfneg = df.loc[~((df.patient_id.isin(dfpostest['patient_id'])))]
dfneg.head(3)

In [ ]:
dfneg.shape

In [ ]:
# Erstellen eines Dataframes von Patienten mit der Nebendiagnose Covid19 
dfNebendiag = pd.read_sql_query("""
select patient_id, CODE, DESCRIPTION from dimConditions where code in ("840539006")
;""", conn_dwh
  )


In [ ]:
dfNebendiag.head(3)

In [ ]:
# Um Sicherzugehen, dass wirklich nur Covid-neg. Patienten in der Kontrollgruppe sind werden auch Patienten ohne Testergebnis aber mit Nebendiagnose Covid19 bzw. Verdacht auf Covid19 ausgeschlossen.
dfneg = dfneg.loc[~((dfneg.patient_id.isin(dfNebendiag["PATIENT_ID"])))]


In [ ]:
# Hinzufügen Spalte mit Wert "0"
dfneg['Covid'] = '0'
dfneg.head(3)

In [ ]:
dfneg.shape

In [ ]:
#Übersicht Kontrolgruppe
dfneg.nunique(axis=0)

### Zusammenfügen von COVID-pos. Kohorte und Kontrollgruppe in einen Dataframe

In [ ]:
dffinal = pd.concat([dfpos, dfneg])
dffinal.head(3)

In [ ]:
# Löschen der COVID-Testergebnisse, damit der Datentyp der Werte der Spalte "Value" in Floats geändert werden kann (Covid-Status ist trotzdem in Spalte Covid festgehalten)
dffinal = dffinal.loc[(dffinal["CODE"] != "94531-1")]


# Konvertieren der Laborwerte und des Covid-Status in Zahlenwerte (Floats)
dffinal["Covid"] = dffinal["Covid"].astype("int")
dffinal["VALUE"] = dffinal["VALUE"].astype("float")

dffinal.head(3)

In [ ]:
#Konvertieren der Datumformats 
dffinal["DATE"] = pd.to_datetime(dffinal["DATE"])
dffinal["DATE"] = dffinal["DATE"].dt.to_period('d')

In [ ]:
dffinal.head()

## Vergleichende Analyse der Laborparameter zwischen den Gruppen

In [ ]:
%matplotlib inline
#Plotting der Nierenparameter
GFR= dffinal[dffinal['CODE'] == "33914-3"]
Urea= dffinal[dffinal['CODE'] == "3094-0"]
Protein= dffinal[dffinal['CODE'] == "2885-2"]

g = sns.catplot(data=GFR, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=GFR, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('GFR', weight='bold').set_fontsize('16')

g = sns.catplot(data=Urea, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Urea, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Urea', weight='bold').set_fontsize('16')

g = sns.catplot(data=Protein, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Protein, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Protein', weight='bold').set_fontsize('16')

In [ ]:
#Plotting der Kreislaufparameter
Temperatur= dffinal[dffinal['CODE'] == "8310-5"]
O2= dffinal[dffinal['CODE'] == "2708-6"]
AF= dffinal[dffinal['CODE'] == "9279-1"]
HF= dffinal[dffinal['CODE'] == "8867-4"]
Syst= dffinal[dffinal['CODE'] == "8480-6"]
Dia= dffinal[dffinal['CODE'] == "8462-4"]

g = sns.catplot(data=Temperatur, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Temperatur, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Körpertemperatur', weight='bold').set_fontsize('16')

g = sns.catplot(data=O2, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=O2, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('O2-Sättigung', weight='bold').set_fontsize('16')

g = sns.catplot(data=AF, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=AF, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Atemfrequenz', weight='bold').set_fontsize('16')

g = sns.catplot(data=HF, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=HF, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Herzfrequenz', weight='bold').set_fontsize('16')

g = sns.catplot(data=Syst, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Syst, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Systolischer Blutdruck', weight='bold').set_fontsize('16')

g = sns.catplot(data=Dia, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Dia, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Diastolischer Blutdruck', weight='bold').set_fontsize('16')


In [ ]:
#Plotting der Leberparameter
ALT= dffinal[dffinal['CODE'] == "1742-6"]
AST= dffinal[dffinal['CODE'] == "1920-8"]
AP= dffinal[dffinal['CODE'] == "6768-6"]
Bili= dffinal[dffinal['CODE'] == "1975-2"]

g = sns.catplot(data=ALT, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=ALT, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('ALT', weight='bold').set_fontsize('16')

g = sns.catplot(data=AST, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=AST, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('AST', weight='bold').set_fontsize('16')

g = sns.catplot(data=AP, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=AP, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('AP', weight='bold').set_fontsize('16')

g = sns.catplot(data=Bili, x="Covid", y="VALUE", kind="violin", inner="quart", linewidth=1)
sns.swarmplot(data=Bili, x="Covid", y="VALUE", color="k", size=2, ax=g.ax)
plt.title('Bilirubin total', weight='bold').set_fontsize('16')

## Vorbereitung der Daten für den Classifier

In [ ]:
# import python packages for ML models
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
dfpred = dffinal.pivot_table('VALUE', ['patient_id',"DATE", "Covid"], 'DESCRIPTION')
dfpred.head(3)

In [ ]:
dfpred = dfpred.drop(["Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma", "Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma", "Bilirubin.total [Mass/volume] in Serum or Plasma", 'Body temperature', "Diastolic Blood Pressure", "Estimated Glomerular Filtration Rate", "Oxygen [Partial pressure] in Arterial blood", "Oxygen saturation in Arterial blood","Protein [Mass/volume] in Serum or Plasma","Systolic Blood Pressure", "Urea nitrogen [Mass/volume] in Serum or Plasma" ], axis=1)
dfpred = dfpred.reset_index()
dfpred.rename(columns={"Glomerular filtration rate/1.73 sq M.predicted": 'GFR', "Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma":  "ALT", "Heart rate": "Heart_rate", "Respiratory rate": "Respiratory_rate"}, inplace=True)
dfpred.tail(3)

In [ ]:
dfpred= dfpred.dropna(thresh=4)

In [ ]:
dfpred.groupby(["Covid"]).size()

In [ ]:
#Exkurs: Fallenlassen von Zeilen mit NaN-Werten reduziert die Kontrollgruppe in nicht tolerierbarem Maße (d.h. die vier Parameter wurden so gut wie nie gleichzeitig bestimmt)
dfpnull = dfpred.dropna()
dfpnull.groupby(["Covid"]).size()

#Es muss daher ein Algorithmus zur Vorhersage verwendet werden, der NaN-Werte akzeptiert --> XGBoost

In [ ]:
# Splitten der Daten in train und test Dataframes
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# Ausgabe der Shapes von train und test Dataframes
dfpred_train, dfpred_test = train_test_split(dfpred, test_size=test_ratio, random_state=0)
dfpred_train.shape, dfpred_test.shape

In [ ]:
# Bestimmen der Features und des Zielwertes
X = dfpred_train.drop(["patient_id", "Covid","DATE"], axis=1)
y = dfpred_train["Covid"]

# Zusätzlich Erstellen eines Validation Dataframes
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_ratio, random_state=0)

X_train.shape, X_val.shape

## Vorhersage mit XGBoost

In [ ]:
# Erstellen des Parameter Grid Search
param_grid = {
    'learning_rate': [0.05, 0.5], #so called `eta` value    
    'max_depth': [2, 5, 10, 15],
    'colsample_bytree': [1],
    # "objective" :["reg:tweedie"],
    'gamma':[0],               
    'min_child_weight':[1.5],
    'n_estimators' : [5, 10],                                                                    
    'reg_alpha': [0.75],
    'reg_lambda' : [0.45],
    'subsample' : [1],
    'random state' : [1]
}
   
# Erstellen des Models
xgb = XGBClassifier()
# Initiieren des Grid Searches
grid_search_xgb = GridSearchCV(estimator = xgb, 
                               param_grid=param_grid,
                               cv=3,
                               n_jobs=-1,
                               verbose=2)

In [ ]:
# Anpassen der Daten an den Grid Search
grid_search_xgb.fit(X_train, y_train)

In [ ]:
# Beste Parameter nach Grid Search
grid_search_xgb.best_params_

In [ ]:
# Beste Model
best_grid_xgb = grid_search_xgb.best_estimator_

# Evaluiierung für die Train Daten
pred_train_xgb = best_grid_xgb.predict(X_train)
print('Accuracy for train data:', accuracy_score(y_train, pred_train_xgb))  

# Evaluiierung für die Test Daten
pred_test_xgb = best_grid_xgb.predict(X_val)
acc_xgb = round(accuracy_score(y_val, pred_test_xgb), 5)
print('Accuracy for test data:', acc_xgb)

In [ ]:
# Hinzufügen des Prediction Ergebnisses zur Ausgangstabelle
selected_columns = X.columns
x_test = dfpred_test[X.columns]
dfpred_test["XGB_prediction"] = best_grid_xgb.predict(x_test)
dfpred_test.head(5)


## Auswertung der Vorhersagen

In [ ]:
# Erstellen einer Confusion-Matrix
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

y_actu = dfpred_test["Covid"]
y_pred = dfpred_test["XGB_prediction"]
cf_matrix = confusion_matrix(y_actu, y_pred)

classes = ['class A', 'class B']
figure, ax = plot_confusion_matrix(conf_mat = cf_matrix,
                                   show_absolute = True,
                                   show_normed = True,
                                   colorbar = True)
plt.show()

In [ ]:
# Darstellen der inkorrekt klassifizierten Daten
incorrect = dfpred_test[dfpred_test["Covid"] != dfpred_test["XGB_prediction"]]
incorrect.head(30)

In [ ]:
# Darstellen der Feature importance
from xgboost import plot_importance
from matplotlib import pyplot
plot_importance(best_grid_xgb)
pyplot.show()

In [ ]:
# Beispielhaftes Darstellen eines Decision Trees
from xgboost import plot_tree
plot_tree(best_grid_xgb, num_trees=0)
fig = plt.gcf()
fig.set_size_inches(50, 30)